In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Causal_Inference_Model/

/content/drive/MyDrive/Causal_Inference_Model


In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Import Libraries

In [ ]:
import dvc.api
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import sys, os

In [45]:
sys.path.append(os.path.abspath('scripts/'))

In [56]:
from filehundle import LoadData
from plots import Plot
from overview import Overview
from preprocessing import PreProcess

Initialize Loaded Data

In [58]:
loader = LoadData()
plot = Plot()

INFO:logger:Successfully Instantiated load_data Class Object
INFO:logger:Successfully Instantiated Preprocessing Class Object


In [59]:
# Get URL from DVC
data_path = 'data/nb.csv'
repo = 'https://github.com/yonamg/Causal_Inference_Model/'
version = '31a6d9ca5315cecd1bf0cfee00a7e0'

In [ ]:
data_url = dvc.api.get_url(
    path=data_path,
    repo=repo,
    rev=version
)

In [62]:
trip_df = loader.read_csv("data/nb.csv")


In [63]:
trip_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536020 entries, 0 to 536019
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Trip ID           536020 non-null  int64 
 1   Trip Origin       536020 non-null  object
 2   Trip Destination  536020 non-null  object
 3   Trip Start Time   534369 non-null  object
 4   Trip End Time     536019 non-null  object
dtypes: int64(1), object(4)
memory usage: 20.4+ MB
